# Hotel Booking

En este notebook uso mi dataset limpio y la configuración "perfecta" para mi pipeline

## <p style="color: #FFFFFF;background: #007E4A80; text-align: center; padding:5px 0; font-weight: 400">Sobre el dataset</p>

>[Fue sacado de este dataset de kaggle](https://www.kaggle.com/datasets/mojtaba142/hotel-booking)

*(Esto lo comencé gracias a un curso de código facilito y me gustó su descripción y diccionario de datos, así que los voy a usar)*<br>
Hotel Facilito se está preparando para las vacaciones de verano, sin embargo, les preocupa que muchos de sus clientes cancelan de ultima hora, dejándoles con habitaciones vacías – sucede que algunos clientes simplemente no se presentan y, a pesar de que a veces se cobra una cuota de reservación, la gran mayoría de las ganancias se obtiene cuando los huéspedes pagan el resto al ocupar la habitación.

Usando sus datos, les gustaría que les ayudaras a identificar a aquellos clientes que tienen más posibilidad de cancelar ya que les gustaría darles seguimiento para que si en caso de que requieran cancelar, se haga con la mayor antelación posible.

 - `hotel`: Hotel en el que se hizo la reserva.
  - `is_canceled`: Indica si la reserva fue cancelada o no.
  - `lead_time`: Número de días que transcurrieron entre la fecha de ingreso de la reserva en el PMS y la fecha de llegada.
  - `arrival_date_year`: Año de la fecha de llegada.
  - `arrival_date_month`: Mes de la fecha de llegada con 12 categorías: "Enero" a "Diciembre".
  - `arrival_date_week_number`: Número de semana de la fecha de llegada.
  - `arrival_date_day_of_month`: Día del mes de la fecha de llegada.
  - `stays_in_weekend_nights`: Número de noches de fin de semana (sábado o domingo) que el huésped se quedó o reservó para quedarse en el hotel.
  - `stays_in_week_nights`: Número de noches de semana (lunes a viernes) que el huésped se quedó o reservó para quedarse en el hotel.
  - `adults`: Número de adultos.
  - `children`: Número de niños.
  - `babies`: Número de bebés.
  - `meal`: Tipo de comida reservada. Valor categórico.
  - `country`: País de origen. Las categorías se representan según ISO 3155–3:2013.
  - `market_segment`: Designación del segmento de mercado.
  - `distribution_channel`: Canal de distribución de la reserva.
  - `is_repeated_guest`: Valor que indica si el nombre de la reserva era de un huésped recurrente (1) o no (0).
  - `previous_cancellations`: Número de reservas anteriores que fueron canceladas por el cliente antes de la reserva actual.
  - `previous_bookings_not_canceled`: Número de reservas anteriores que no fueron canceladas por el cliente antes de la reserva actual.
  - `reserved_room_type`: Código del tipo de habitación reservada. El código se presenta en lugar de la designación por razones de anonimato.
  - `assigned_room_type`: Código del tipo de habitación asignada a la reserva. A veces, el tipo de habitación asignada difiere del tipo de habitación reservada debido a razones operativas del hotel (por ejemplo, sobreventa) o por solicitud del cliente. El código se presenta en lugar de la designación por razones de anonimato.
  - `booking_changes`: Número de cambios/modificaciones realizados a la reserva desde el momento en que se ingresó en el PMS hasta el momento del check-in o la cancelación.
  - `deposit_type`: Indicación de si el cliente hizo un depósito para garantizar la reserva.
  - `agent`: ID de la agencia de viajes que realizó la reserva.
  - `company`: ID de la empresa/entidad que realizó la reserva o es responsable de pagar la reserva.
  - `days_in_waiting_list`: Número de días que la reserva estuvo en lista de espera antes de ser confirmada al cliente.
  - `customer_type`: Tipo de reserva.
  - `adr`: Tarifa diaria promedio.
  - `required_car_parking_spaces`: Número de espacios de estacionamiento requeridos por el cliente.
  - `total_of_special_requests`: Número de solicitudes especiales realizadas por el cliente (por ejemplo, cama doble o piso alto).
  - `reservation_status`: Último estado de la reserva.
  - `reservation_status_date`: Fecha en que se estableció el último estado.
  - `name`: Nombre del cliente.
  - `email`: Correo electrónico del cliente.
  - `phone`: Teléfono del cliente.
  - `credit_card`: Últimos cuatro dígitos de la tarjeta de crédito del cliente.

## <p style="color: #FFFFFF;background: #007E4A80; text-align: center; padding:5px 0; font-weight: 400"> Pipelines </p>

Básicamente lo hago con pipelines para poder sacarlo a producción, "creo"

In [15]:
import pandas as pd

df = pd.read_csv('data/hotel_booking_clean.csv')

In [16]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.preprocessing import OneHotEncoder

### <p style="color: #FFFFFF;background: #006B8380; text-align: center; padding:5px 0; font-weight: 400"> Train test split </p>

In [17]:
X = df.drop(['is_canceled'], axis=1)
y = df['is_canceled']

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_rest, y_train, y_rest = train_test_split(X, y, test_size=0.4, random_state=42,stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_rest, y_rest, test_size=0.5, random_state=42,stratify=y_rest)

print("Tamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de validación:", X_val.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)

Tamaño del conjunto de entrenamiento: (61631, 31)
Tamaño del conjunto de validación: (20544, 31)
Tamaño del conjunto de prueba: (20544, 31)


### <p style="color: #FFFFFF;background: #006B8380; text-align: center; padding:5px 0; font-weight: 400"> Feature Engineering </p>

In [19]:
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import RobustScaler

# Cuando se hace un modelo para "producción" es necesario usar one hot y no dummies
one_hot_encoding = ColumnTransformer([
    ('one_hot_encode',
    OneHotEncoder(sparse_output=False, handle_unknown='ignore'),
    ['hotel','meal','market_segment','assigned_room_type','deposit_type','customer_type','arrival_date_month'])
])

binarizer = ColumnTransformer([
    (
        'binarizer',
        Binarizer(),        
        ['previous_cancellations','previous_bookings_not_canceled','booking_changes','required_car_parking_spaces','total_of_special_requests']        
    )
])

# one_hot_binarized = Pipeline([
#     ('binarizer',binarizer),
#     ('one_hot_encoder',OneHotEncoder(sparse_output=False,handle_unknown='ignore'))
# ])

passthrough = ColumnTransformer([
    ('passthrough',
    'passthrough',
    ['lead_time', 'arrival_date_year', 'arrival_date_week_number',
    'arrival_date_day_of_month', 'stays_in_weekend_nights',
    'stays_in_week_nights', 'adults', 'children', 'babies',
    'is_repeated_guest', 'days_in_waiting_list','adr'])
])

In [20]:
from sklearn.pipeline import FeatureUnion
from sklearn.pipeline import Pipeline

feature_ingineering_pipe = Pipeline([
        (
            "features",
            FeatureUnion(
                [
                    ("encoding", one_hot_encoding),
                    ("binarized", binarizer),
                    # ("encoding_binarized", one_hot_binarized),
                    ("pass", passthrough)
                ]
            ),
        )
])

Esto es unicamente como para probar como quedó nuestro df luego del feature_engineering

In [21]:
transformed = feature_ingineering_pipe.fit_transform(X_train)
column_names = feature_ingineering_pipe.named_steps['features'].get_feature_names_out()
transformed_df = pd.DataFrame(transformed, columns=column_names)

transformed_df.shape

(61631, 60)

### <p style="color: #FFFFFF;background: #006B8380; text-align: center; padding:5px 0; font-weight: 400"> Model</p>

In [22]:
from sklearn.base import clone
from sklearn.ensemble import RandomForestClassifier

model_pipe = Pipeline([
        ('Feature_engineering',clone(feature_ingineering_pipe)),
        ('model',RandomForestClassifier(n_estimators=99,random_state=10, criterion='entropy',max_depth=10))
])

### <p style="color: #FFFFFF;background: #006B8380; text-align: center; padding:5px 0; font-weight: 400"> Model testing</p>

Ahora igual que arriba, uno los train_x con validate_x y pruebo mi modelo

In [23]:
df_x = pd.concat([X_train,X_val])
df_y = pd.concat([y_train, y_val])

Ahora si podemos usar todo

In [24]:
model_pipe.fit(df_x,df_y)

Pipeline(steps=[('Feature_engineering',
                 Pipeline(steps=[('features',
                                  FeatureUnion(transformer_list=[('encoding',
                                                                  ColumnTransformer(transformers=[('one_hot_encode',
                                                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                                                 sparse_output=False),
                                                                                                   ['hotel',
                                                                                                    'meal',
                                                                                                    'market_segment',
                                                                                                    'assigned_room_type',
                                                                                                    'deposit_type',
                                                                                                    'customer_type',
                                                                                                    'arrival_date_month'])])),
                                                                 ('binarized',
                                                                  ColumnT...
                                                                  ColumnTransformer(transformers=[('passthrough',
                                                                                                   'passthrough',
                                                                                                   ['lead_time',
                                                                                                    'arrival_date_year',
                                                                                                    'arrival_date_week_number',
                                                                                                    'arrival_date_day_of_month',
                                                                                                    'stays_in_weekend_nights',
                                                                                                    'stays_in_week_nights',
                                                                                                    'adults',
                                                                                                    'children',
                                                                                                    'babies',
                                                                                                    'is_repeated_guest',
                                                                                                    'days_in_waiting_list',
                                                                                                    'adr'])]))]))])),
                ('model',
                 RandomForestClassifier(criterion='entropy', max_depth=10,
                                        n_estimators=99, random_state=10))])

In [26]:
from sklearn.metrics import accuracy_score, recall_score

test_pred_y = model_pipe.predict(X_test)
print(accuracy_score(test_pred_y, y_test), recall_score(test_pred_y, y_test))

0.8016452492211839 0.8870999410956214


## <p style="color: #FFFFFF;background: #007E4A80; text-align: center; padding:5px 0; font-weight: 400"> Model persistence </p>

In [27]:
from joblib import dump, load

dump(model_pipe, 'modelo_pipeline.joblib')

['modelo_pipeline.joblib']

In [28]:
modelo_rfc_pipeline = load('modelo_pipeline.joblib')

new_customers = pd.read_csv('data/new_customers.csv')
new_customers['will_cancel'] = modelo_rfc_pipeline.predict(new_customers)
new_customers[['proba_check_in', 'proba_cancel']] = modelo_rfc_pipeline.predict_proba(new_customers)

In [29]:
new_customers[['name', 'phone-number', 'will_cancel', 'proba_cancel']].sort_values(by='proba_cancel', ascending=False).head(20)

,name,phone-number,will_cancel,proba_cancel
26,Justin Taylor,335-560-0061,1,0.994497
90,Regina Pacheco,350-100-9605,1,0.993910
64,Daniel Ortiz,960-672-0720,1,0.993076
28,James Henson,161-506-7759,1,0.991824
58,Cory Alexander,864-688-3246,1,0.990360
46,Katelyn Jones,323-339-3265,1,0.984576
94,Dennis Reyes,398-564-0923,1,0.984333
72,Thomas Mills,756-971-1624,1,0.977496
43,Joseph Lawson,166-493-3428,1,0.971034
8,Carrie Tanner,392-436-1692,1,0.969604
